In [35]:
# !pip install ta

In [36]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ta

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
from pmdarima.arima import auto_arima
from pylab import rcParams
import requests
import json
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from xgboost import XGBRegressor

In [37]:
msft = yf.Ticker("MSFT")
msft_df = msft.history(
    period="max", 
    interval="1h",
)

print(msft_df.head())

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-06-20 09:30:00-04:00  339.695801  342.079987  336.239990  336.380005   
2023-06-20 10:30:00-04:00  336.399994  337.799988  335.859985  336.940002   
2023-06-20 11:30:00-04:00  337.000000  338.565002  336.054901  338.230011   
2023-06-20 12:30:00-04:00  338.239990  339.698608  338.190002  339.095001   
2023-06-20 13:30:00-04:00  339.109985  339.320007  338.246887  339.130005   

                            Volume  Dividends  Stock Splits  
Datetime                                                     
2023-06-20 09:30:00-04:00  6650745        0.0           0.0  
2023-06-20 10:30:00-04:00  3641713        0.0           0.0  
2023-06-20 11:30:00-04:00  2611871        0.0           0.0  
2023-06-20 12:30:00-04:00  1933145        0.0           0.0  
2023-06-20 13:30:00-04:00  1812231        0.0           0.0  


google sites or sights

## Feature Engineering

### Lagged Prices

In [38]:
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    lag_periods = [1, 2, 3, 4, 5, 6] # within 1 day range
    for lag in lag_periods:
        msft_df[f'{col}_price_lag_{lag}H'] = msft_df[col].shift(lag)

### Lagged Returns

#### Hourly

In [39]:
msft_df['Hourly_return'] = msft_df['Close'].pct_change(periods=1)
lag_returns_hourly = [1, 2, 3, 4, 5, 6] # range within 1 day
for lag in lag_returns_hourly:
    msft_df[f'Hourly_return_lag_{lag}H'] = msft_df['Hourly_return'].shift(lag)

#### Daily

In [40]:
msft_df['Daily_return'] = msft_df['Close'].pct_change(periods=7)
lag_returns_daily = [7,14,21,28] # 1-4 day range
for lag in lag_returns_daily:
    msft_df[f'Daily_return_lag_{lag}H'] = msft_df['Daily_return'].shift(lag)

#### Weekly

In [41]:
msft_df['Weekly_return'] = msft_df['Close'].pct_change(periods=35)
lag_returns_weekly = [35,70,105] # range within 3 weeks
for lag in lag_returns_weekly:
    msft_df[f'weekly_return_lag_{lag}H'] = msft_df['Weekly_return'].shift(lag)

### Moving Averages

#### Simple

In [42]:
sma_windows = [5, 7, 10, 21, 35, 50, 105, 140]
for window in sma_windows:
    msft_df[f'SMA_{window}H'] = msft_df['Close'].rolling(window=window).mean()

#### Exponential

In [43]:
ema_windows = [5, 7, 10, 21, 35, 50, 105, 140]
for window in ema_windows:
    msft_df[f'EMA_{window}H'] = msft_df['Close'].ewm(span=window, adjust=False).mean()

#### Golden Crosses

Using EMA for intra-day pricing. If using daily, weekly, monthly, SMA is preferred method.

In [44]:
msft_df['Golden_Cross'] = ((msft_df['EMA_21H'] > msft_df['EMA_50H']) & (msft_df['EMA_21H'].shift(1) <= msft_df['EMA_50H'].shift(1))).astype(int)

#### Death Crosses

In [45]:
msft_df['Death_Cross'] = ((msft_df['EMA_21H'] < msft_df['EMA_50H']) & (msft_df['EMA_21H'].shift(1) <= msft_df['EMA_50H'].shift(1))).astype(int)

### Volatility

#### Standard Deviation

Use returns rather than raw price for STD due to returns having stationarity.

In [46]:
volatility_window = [3, 7, 14, 35]

for window in volatility_window:
    msft_df[f'Volatility_StdDEV_{window}H'] = msft_df['Hourly_return'].rolling(window=window).std()

#### Average True Range (ATR)

The 'true' stock prices volatility over the window using EMA.

In [47]:
atr_periods = [7, 14, 35, 70]
for period in atr_periods:
    msft_df[f'ATR_{period}H'] = ta.volatility.average_true_range(
        high = msft_df['High'],
        low = msft_df['Low'],
        close = msft_df['Close'],
        window=period
    )

In [ ]:
fill with most recent value (next hour, current, past)
bbfill ffill, sideway fill? -rolling average of the most recent to fill nans

In [49]:
msft_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_price_lag_1H,Open_price_lag_2H,Open_price_lag_3H,...,Golden_Cross,Death_Cross,Volatility_StdDEV_3H,Volatility_StdDEV_7H,Volatility_StdDEV_14H,Volatility_StdDEV_35H,ATR_7H,ATR_14H,ATR_35H,ATR_70H
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-06-20 09:30:00-04:00,339.695801,342.079987,336.239990,336.380005,6650745,0.0,0.0,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2023-06-20 10:30:00-04:00,336.399994,337.799988,335.859985,336.940002,3641713,0.0,0.0,339.695801,NaN,NaN,...,1,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2023-06-20 11:30:00-04:00,337.000000,338.565002,336.054901,338.230011,2611871,0.0,0.0,336.399994,339.695801,NaN,...,0,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2023-06-20 12:30:00-04:00,338.239990,339.698608,338.190002,339.095001,1933145,0.0,0.0,337.000000,336.399994,339.695801,...,0,0,0.001087,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
2023-06-20 13:30:00-04:00,339.109985,339.320007,338.246887,339.130005,1812231,0.0,0.0,338.239990,337.000000,336.399994,...,0,0,0.001894,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-17 11:30:00-04:00,476.839996,478.040009,476.734802,477.679993,1186453,0.0,0.0,476.122009,475.350006,478.510010,...,0,0,0.004425,0.002910,0.003862,0.003693,2.054859,2.093077,2.100867,2.252786
2025-06-17 12:30:00-04:00,477.641815,478.739899,476.614990,476.614990,1104756,0.0,0.0,476.839996,476.122009,475.350006,...,0,0,0.002220,0.002905,0.003796,0.003721,2.064866,2.095351,2.101554,2.250960
2025-06-17 13:30:00-04:00,476.589996,477.269989,475.390015,476.435089,1220898,0.0,0.0,477.641815,476.839996,476.122009,...,0,0,0.001997,0.002746,0.003678,0.003683,2.038453,2.079967,2.095223,2.245660


In [48]:
# original_rows = len(msft_df)
# msft_df.dropna(inplace=True)
# print(f"\nDropped {original_rows - len(msft_df)} rows due to NaN values after feature engineering.")

# print("\n--- Final DataFrame Head with Engineered Features (showing new return features) ---")
# #print(msft_df[['Close', 'Hourly_Return', 'Daily_Return', 'Weekly_Return',
# #'Hourly_Return_lag_1H', 'Daily_Return_lag_24H', 'Weekly_Return_lag_168H']].head(10))
# #print("\n--- Final Dat/aFrame Info with Engineered Features ---")
# msft_df.info()
# print("\n--- Columns in the Final DataFrame ---")
# print(msft_df.columns.tolist())